# Text Generation using GPT (Using Huggingface)

## Project Setup

In [ ]:
!pip install -q transformers

In [ ]:
import torch
import shutil
from torch.utils.data import Dataset, random_split
from transformers import Trainer, TrainingArguments, GPTNeoForCausalLM, GPT2Tokenizer


from google.colab import drive


## Data Preparation

In [ ]:
# Load data into colab
!wget https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt

--2023-03-24 21:15:41--  https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94275 (92K) [text/plain]
Saving to: ‘shakespeare.txt.2’

shakespeare.txt.2   100%[===================>]  92.07K  --.-KB/s    in 0.001s  

2023-03-24 21:15:41 (69.2 MB/s) - ‘shakespeare.txt.2’ saved [94275/94275]



In [ ]:
# Connects colab to google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shutil.copy("/content/shakespeare.txt.2","drive/MyDrive/AICamp/nlp")


In [ ]:
# Read the text file and returns list of lines in text
def read_file(file_path):
    with open(file_path) as f:
        lines = [line for line in f]
        # lines.remove("")
    return lines


In [ ]:
file_path = "drive/MyDrive/AICamp/nlp/shakespeare.txt.2"

texts = read_file(file_path)
sonnets = []
sonnet = []
for text in texts:
  if len(text)>1:
    sonnet.append(text)
  else:
    sonnets.append(''.join(sonnet))
    sonnet = []

# Remove unnecessary texts


'THE SONNETS\n'

In [ ]:
# Prepare sonnets
datas = sonnets[2:-1]
print(len(datas))
for data in datas:
  if len(data)<1:
    datas.remove(data)
print(len(datas))

289
212


In [ ]:
# Custome dataset class to load dataset
class ShakespeareDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer('<|startoftext|>' 
                                        + txt +    
                                        '<|endoftext|>',
                                        truncation=True,
                                        max_length=max_length, 
                                            padding="max_length")
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

## Initialize tokenizer, model

In [ ]:
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)

# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end 
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token='<|startoftext|>',
                            eos_token='<|endoftext|>',
                            pad_token='<|pad|>')

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()

# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

## Train/Test Split data

In [ ]:
max_length = max([len(tokenizer.encode(sonnet)) for sonnet in datas])

# Load dataset
dataset = ShakespeareDataset(sonnets, tokenizer, max_length)

# Split data into train/val
train_size = int(0.9 * len(dataset))

train_data, val_data = random_split(dataset, [train_size, len(dataset) - train_size])

max_length

351

In [ ]:
tokenizer.batch_decode(val_data[0])

["<|startoftext|> When to the sessions of sweet silent thought,\nI summon up remembrance of things past,\nI sigh the lack of many a thing I sought,\nAnd with old woes new wail my dear time's waste:\nThen can I drown an eye (unused to flow)\nFor precious friends hid in death's dateless night,\nAnd weep afresh love's long since cancelled woe,\nAnd moan th' expense of many a vanished sight.\nThen can I grieve at grievances foregone,\nAnd heavily from woe to woe tell o'er\nThe sad account of fore-bemoaned moan,\nWhich I new pay as if not paid before.\nBut if the while I think on thee (dear friend)\nAll losses are restored, and sorrows end.\n  \nThy bosom is endeared with all hearts,\nWhich I by lacking have supposed dead,\nAnd there reigns love and all love's loving parts,\nAnd all those friends which I thought buried.\nHow many a holy and obsequious tear\nHath dear religious love stol'n from mine eye,\nAs interest of the dead, which now appear,\nBut things removed that hidden in thee lie.

## Train Model

In [ ]:
# Here I will pass the output directory where 
# the model predictions and checkpoints will be stored, 
# batch sizes for the training and validation steps, 
# and warmup_steps to gradually increase the learning rate
learning_rates = [5e-5, 3e-5, 1e-5]

for learning_rate in learning_rates:

    training_args = TrainingArguments(output_dir=f'./results_{learning_rate}',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=1000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,                               
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=learning_rate,
                                      weight_decay=0.01,  
                                      logging_dir=f'./logs_{learning_rate}')

    trainer = Trainer(model=model, args=training_args,  
                      train_dataset=train_data,
                      eval_dataset=val_data, 
                      # This custom collate function is necessary 
                      # to built batches of data
                      data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),       
                  'attention_mask': torch.stack([f[1] for f in data]),
                  'labels': torch.stack([f[0] for f in data])})

    # Start training process!
    print(f"Training result for learning rate: {learning_rate}")
    trainer.train()
    print("\n\n")

Training result for learning rate: 5e-05


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.648800,4.627812
2000,0.614800,4.719461
3000,0.589600,4.778398
4000,0.560300,4.772144
5000,0.528100,4.829613





Training result for learning rate: 3e-05


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.546000,4.813454
2000,0.545200,4.786867
3000,0.537200,4.869326
4000,0.524900,4.870878
5000,0.506800,4.928403





Training result for learning rate: 1e-05


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.502200,4.942055
2000,0.499100,4.859634
3000,0.499400,4.882948
4000,0.495000,4.956717
5000,0.487600,4.955092


BAsed on the results above, it looks like model trained with learning rate = 5e-5 is more promising than others.

In [ ]:
training_args = TrainingArguments(output_dir=f'./results',
                                      num_train_epochs=5,
                                      logging_steps=1000,
                                      save_steps=5000,
                                      evaluation_strategy='steps',
                                      eval_steps=1000,                               
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      warmup_steps=100,
                                      learning_rate=5e-5,
                                      weight_decay=0.01,  
                                      logging_dir=f'./logs')

trainer = Trainer(model=model, args=training_args,  
                  train_dataset=train_data,
                  eval_dataset=val_data, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),       
              'attention_mask': torch.stack([f[1] for f in data]),
              'labels': torch.stack([f[0] for f in data])})

# Start training process!
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=655, training_loss=1.0412405698354008, metrics={'train_runtime': 163.5666, 'train_samples_per_second': 8.009, 'train_steps_per_second': 4.004, 'total_flos': 234581319198720.0, 'train_loss': 1.0412405698354008, 'epoch': 5.0})

In [ ]:
# Save model in the specified file path
trainer.save_model("drive/MyDrive/AICamp/nlp/")

In [ ]:
tokenizer.save_pretrained("drive/MyDrive/AICamp/nlp/")

('drive/MyDrive/AICamp/nlp/tokenizer_config.json',
 'drive/MyDrive/AICamp/nlp/special_tokens_map.json',
 'drive/MyDrive/AICamp/nlp/vocab.json',
 'drive/MyDrive/AICamp/nlp/merges.txt',
 'drive/MyDrive/AICamp/nlp/added_tokens.json')

## Checking Model Output

In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                # bos_token='<|startoftext|>',
                                # eos_token='<|endoftext|>', pad_token='<|pad|>',
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: 
1: 
2: Where should that evil cease, or those things would return to their owners'?
To prove their good grace (That thou must find worthy still remains unhearest, thou must seek this end, and that this which thine (Nor if this be so thy worst must be my joy)?
Let me return not once nor hold still an unseeing eye
A pity where's this coming help bestow thee?
Then thine eye be thy back-ticks where it most hinds theego
Where reason guides both minds whence thou wost and me to seek!?


3: In the earth it holds but gentleclips and jewels,
Lest things that belong alone, should themselves boast,
The precious parts so grounded to nature'er line,   
Without having checked this abundance themselves bear,   
SIXth many miles in motion from the desert place.   
Nor are stars in all kinds of stars less grounded,
They dwell on me by giving account to my skill:
And though some are not more great or more than others,
More even or even all their compass height than others.
All this hath in the world

In [ ]:
! transformers-cli env

2023-03-24 22:53:44.974369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 22:53:44.974575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-24 22:53:44.974603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2023-03-24 22:53:55.043416: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:

## Upload model to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

In [ ]:
# Create your repo first to upload the model
api.create_repo(repo_id="gpt2-sonnet-generators")

RepoUrl('https://huggingface.co/niki-stha/gpt2-sonnet-generators', endpoint='https://huggingface.co', repo_type='model', repo_id='niki-stha/gpt2-sonnet-generators')

In [ ]:
# Upload your model to huggingface. You can clone the repo anytime to use the model.
import os

model_pth = "drive/MyDrive/AICamp/nlp/models"

files = os.listdir(model_pth)
for fi in files:
    print(os.path.join(model_pth, fi))

    api.upload_file(
        path_or_fileobj=os.path.join(model_pth, fi),
        path_in_repo=fi,
        repo_id="niki-stha/gpt2-sonnet-generators",
        repo_type="model",
    )

drive/MyDrive/AICamp/nlp/models/config.json
drive/MyDrive/AICamp/nlp/models/generation_config.json
drive/MyDrive/AICamp/nlp/models/pytorch_model.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

drive/MyDrive/AICamp/nlp/models/training_args.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

drive/MyDrive/AICamp/nlp/models/tokenizer_config.json
drive/MyDrive/AICamp/nlp/models/special_tokens_map.json
drive/MyDrive/AICamp/nlp/models/added_tokens.json
drive/MyDrive/AICamp/nlp/models/vocab.json
drive/MyDrive/AICamp/nlp/models/merges.txt
